In [1]:
from PIL import Image
from torchvision import transforms
import torch


In [3]:
image_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [5]:
def preprocess_image(image_path):
    image = Image.open(image_path).convert("RGB")
    image_tensor = image_transform(image)
    return image_tensor


In [7]:
# Load dataset again (safe practice in each notebook)
import os
import pandas as pd

def load_mvsa_dataset(base_path="MVSA_Single"):
    label_file = os.path.join(base_path, "labelResultAll.txt")
    data_dir = os.path.join(base_path, "data")

    records = []

    with open(label_file, "r", encoding="utf-8", errors="ignore") as f:
        lines = f.readlines()

    for line in lines[1:]:
        parts = line.strip().split("\t")
        if len(parts) != 2:
            continue

        img_id = parts[0]
        sentiments = parts[1].split(",")

        if len(sentiments) != 2:
            continue

        text_sentiment = sentiments[0].strip().lower()
        label_map = {"negative": 0, "neutral": 1, "positive": 2}

        if text_sentiment not in label_map:
            continue

        text_path = os.path.join(data_dir, f"{img_id}.txt")
        image_path = os.path.join(data_dir, f"{img_id}.jpg")

        if not os.path.exists(image_path):
            continue

        records.append({
            "image_path": image_path
        })

    return pd.DataFrame(records)

df_img = load_mvsa_dataset()

# Test on first image
img_path = df_img.iloc[0]["image_path"]
img_tensor = preprocess_image(img_path)

print("Image path:", img_path)
print("Image tensor shape:", img_tensor.shape)
print("Min pixel value:", img_tensor.min().item())
print("Max pixel value:", img_tensor.max().item())


Image path: MVSA_Single/data/1.jpg
Image tensor shape: torch.Size([3, 224, 224])
Min pixel value: -2.1179039478302
Max pixel value: 2.640000104904175
